In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!cp '/content/gdrive/My Drive/FUMPE.zip' '/content'

In [0]:
drive.flush_and_unmount()

In [0]:
 rm -r '/content/dataset/FUMPE/mip'

In [0]:
![ ! -d '/content/dataset_compressed' ] && mkdir '/content/dataset_compressed'
![ ! -d '/content/dataset' ] && mkdir '/content/dataset'
![ ! -d '/content/dataset/FUMPE' ] && mkdir '/content/dataset/FUMPE'
![ ! -d '/content/dataset/FUMPE/mip' ] && mkdir '/content/dataset/FUMPE/mip'
![ ! -d '/content/dataset/FUMPE/mip/train' ] && mkdir '/content/dataset/FUMPE/mip/train'
![ ! -d '/content/dataset/FUMPE/mip/test' ] && mkdir '/content/dataset/FUMPE/mip/test'
![ ! -d '/content/dataset/FUMPE/mip/validate' ] && mkdir '/content/dataset/FUMPE/mip/validate'

In [0]:
!pip install rarfile
!pip install dicom
!pip install scipy

In [0]:
import rarfile
import glob
import dicom
import numpy
import matplotlib.pyplot as plt
import scipy.io as sio

In [0]:
!unzip  FUMPE.zip -d /content/dataset_compressed/FUMPE

In [0]:
files = glob.glob('/content/dataset_compressed/FUMPE/*')
for file in files:
  print("extracting " + file +"  ...")
  rarfile.RarFile(file).extract('/content/dataset/FUMPE')

In [0]:
def mip(array, axis):
  return numpy.amax(array,axis)

In [0]:
def write_dicom(pixel_array,filename):
    import dicom, dicom.UID
    from dicom.dataset import Dataset, FileDataset
    file_meta = Dataset()
    ds = FileDataset(filename, {},file_meta = file_meta)
    ds.PixelData = pixel_array.tostring()
    ds.save_as(filename)
    return

In [0]:
def dcmToNumpyArray(path):
  ds = dicom.read_file(dcmFile)
  return ds.pixel_array

In [0]:
patients = glob.glob('/content/dataset/FUMPE/PAT*')
patient_mip ={}
patientIds = []
for patient_dir in patients:
  dcmFiles = []
  for filePath in glob.glob(patient_dir + '/*.dcm'):
    dcmFiles.append(filePath)
  file = dicom.read_file(dcmFiles[0])
  
  patientId = patient_dir.split("/")[4]
  patientIds.append(patientId)
  print("processing for " + patientId)

  ConstPixelDims = (int(file.Rows), int(file.Columns),int(len(dcmFiles)))

  ConstPixelSpacing = (float(file.PixelSpacing[0]), float(file.PixelSpacing[1]), float(file.SliceThickness))

  ArrayDicom = numpy.zeros(ConstPixelDims, dtype=file.pixel_array.dtype)

  for dcmFile in dcmFiles:
      ArrayDicom[:, :,dcmFiles.index(dcmFile)] = dcmToNumpyArray(dcmFile)
  np_mip = mip(ArrayDicom,2)
  patient_mip[patientId] = np_mip
#   plt.imshow(np_mip, interpolation='nearest')
#   plt.show()
# print(patientIds)

In [0]:
def matToNumpyArray(path, key):
  test = sio.loadmat(path)
  arr = numpy.array(test[key], dtype = numpy.int64)
  return arr

In [0]:
import scipy
for patientId in patientIds:
  data = patient_mip[patientId]
  path = '/content/dataset/FUMPE/mip/' + patientId + '.mat'
  tmp = {"data": data}

  scipy.io.savemat(path, tmp)
  data1 = matToNumpyArray(path, 'data')
#   plt.imshow(data1, interpolation='nearest')
#   plt.show()
